In [128]:
import pandas as pd
import matplotlib

In [129]:
data = pd.read_csv('data/train_data/train_task_3_4.csv')
print(len(data))
data.head()

1382727


,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue
0,898,2111,280203,1,2,2
1,767,3062,55638,1,3,3
2,165,1156,386475,1,2,2
3,490,1653,997498,1,4,4
4,298,3912,578636,1,3,3


In [130]:
# # Confidence based
# answer_metadata = pd.read_csv('data/metadata/answer_metadata_task_3_4.csv')
# data=data.merge(answer_metadata, on="AnswerId", how="left")
# data=data.dropna(subset=["Confidence"])
# data=data.groupby("QuestionId")[["Confidence"]].mean().sort_values("Confidence",ascending=False).reset_index()
# data

# Correct-ratio based
data["CorrectRatio"]=data.groupby('QuestionId')["IsCorrect"].mean()
data=data.dropna()
data=data.sort_values("CorrectRatio",ascending=False).reset_index()
data

,index,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue,CorrectRatio
0,841,823,1996,31280,1,4,4,1.000000
1,847,464,3965,799239,1,2,2,1.000000
2,660,911,6014,690006,0,2,3,1.000000
3,924,451,3907,321824,1,4,4,0.923497
4,422,283,1509,827286,0,2,4,0.895833
...,...,...,...,...,...,...,...,...
943,718,920,5037,409573,1,1,1,0.183891
944,425,25,2284,1167675,1,2,2,0.179487
945,155,884,3014,1328348,0,2,4,0.164776
946,931,326,1717,959796,1,3,3,0.160400


In [131]:
eval_validation = pd.read_csv('data/test_data/quality_response_remapped_public.csv')
print(len(eval_validation))
eval_validation.head()

25


,Unnamed: 0.1,Unnamed: 0,left,right,T1_ALR,T2_CL,T3_GF,T4_MQ,T5_NS
0,0,0,909,318,2,2,2,1.0,2
1,16,16,285,609,2,2,2,2.0,2
2,48,48,132,271,1,2,2,2.0,1
3,38,38,704,127,2,2,1,2.0,2
4,33,33,16,586,1,1,2,2.0,2


In [132]:
eval_test = pd.read_csv('data/test_data/quality_response_remapped_private.csv')
print(len(eval_test))
eval_test.head()

25


,Unnamed: 0.1,Unnamed: 0,left,right,T1_ALR,T2_CL,T3_GF,T4_MQ,T5_NS
0,15,15,145,762,2,2,2,2.0,2
1,31,31,307,208,1,1,1,2.0,1
2,25,25,278,195,1,1,1,1.0,1
3,3,3,13,153,2,2,2,2.0,2
4,40,40,23,100,2,1,1,1.0,1


In [133]:
submission = pd.read_csv('submission/template.csv')
print(len(submission))
print(submission.head())
for idx in range(len(data)):
    tmp = data[data['QuestionId']==idx].index
    submission.loc[idx,'ranking'] = 987654321+idx if tmp.empty else tmp[0]
print(data[710:720])
submission.head()

948
   QuestionId  ranking
0           0      NaN
1           1      NaN
2           2      NaN
3           3      NaN
4           4      NaN
     index  QuestionId  UserId  AnswerId  IsCorrect  CorrectAnswer   
710    632         451    3532    471747          1              4  \
711    215         439    2045    671312          0              2   
712    467         352    3145    365640          0              3   
713    918         572    6048    245431          0              4   
714     84          30    1322    744913          1              1   
715    640         488    3395   1393491          1              3   
716    903         438    1235   1235820          1              1   
717    385         215    4674   1045363          1              2   
718    869         584    4119    169041          0              4   
719     32         404     184    457648          0              2   

     AnswerValue  CorrectRatio  
710            4      0.386861  
711            1     

,QuestionId,ranking
0,0,685.0
1,1,987654322.0
2,2,649.0
3,3,640.0
4,4,203.0


In [134]:
def order2preference(ord, QL, QR):
    preference = []
    for idx in range(len(QL)):
        rankL = ord[ord.QuestionId==QL[idx]].ranking.values[0]
        rankR = ord[ord.QuestionId==QR[idx]].ranking.values[0]
        preference.append(1 if rankL<rankR else 2)
    return preference

def evaluate(truth, submission):
    QL = list(truth.left)
    QR = list(truth.right)
    if len(QL) != len(QR):
        message = 'left and right lengths are not the same'
        exit(message)
    print(list(truth["preference"]))
    return sum(x==y for x,y in zip(truth["preference"],order2preference(submission,QL,QR)))/len(QL)

def evaluates(truth, submission):
    return [
        evaluate(truth.rename(columns={"T1_ALR":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T2_CL":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T3_GF":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T4_MQ":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T5_NS":"preference"})[["left","right","preference"]], submission)]


In [135]:
list_vali = evaluates(eval_validation,submission)
print(list(order2preference(submission,list(eval_validation.left),list(eval_validation.right))))
print(max(list_vali))
print(list_vali)

[2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1]
[2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1]
[2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1]
[1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0]
[2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1]
[2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1]
0.68
[0.64, 0.68, 0.68, 0.56, 0.56]


In [136]:
list_test = evaluates(eval_test,submission)
print(max(list_test))
print(list_test)
print(list(order2preference(submission,list(eval_test.left),list(eval_test.right))))

[2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2]
[2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2]
[2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 2]
[2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 2.0, nan, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 2.0]
[2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2]
0.52
[0.44, 0.48, 0.48, 0.52, 0.36]
[1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1]
